# ASSIGNMENT 0

In [13]:
import cv2
import os
import numpy as np

# 1.1 Video ↔ Images: 
Write a program to convert a given video to its constituent images. Your output should be in a specified folder. 

In [2]:
def videoToImages(input_video_path,output_images_path):
    if not os.path.exists(output_images_path):
        os.makedirs(output_images_path)
    
    # creating object for Capturing video..
    cvideo = cv2.VideoCapture(input_video_path)
    retval, image = cvideo.read()
    count = 0
    print("framing....")
    while retval:
        #getting each frame one by one..
        cv2.imwrite(f"{output_images_path}/frame_{count}.jpg",image)
        count +=1
        retval, image = cvideo.read()
    print("Done.")

In [3]:
# giving input for video and output folder
input_video_path = 'video/sample.mp4'
output_images_path = 'images'
videoToImages(input_video_path,output_images_path)

framing....
Done.


# 1.2 Images ↔ Video: 
Write another program that will merge a set of images in a folder into a single video. 
You should be able to control the frame rate in the video that is created.

In [4]:
def images_to_video(images_folder , output_Dir , frame_rate):
    # get the list of images fro image_folder
    images = [file for file in os.listdir(images_folder) if file.endswith(".jpg")]
    
    images = sorted(images, key = lambda filename : int(filename.split('_')[-1].split('.')[0]))
    #fist image path to get the idea of shape of images.
    first_image_path = os.path.join(images_folder, images[0])
    
    #read the first image  to get height and width..
    first_image = cv2.imread(first_image_path)
    height, width = first_image.shape[:2]
    frameSize = (width, height)
#     print(images)
    
    #get fourcc - > code  which is 4 byte codec for compress the frames.
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    print("fourcc -> ",fourcc)
    #creating a VideoWriter object to create video.
    video_writer  = cv2.VideoWriter(output_Dir ,fourcc , frame_rate, frameSize)
    
    print("writing frames..")
    #writing each image into the video as frames
    for img in images:
        img_path = os.path.join(images_folder,img)
        frame = cv2.imread(img_path)
        video_writer.write(frame)
        
    video_writer.release() # release the resources
    print("done.")

In [5]:
#giving input folder of images and output directory name along with frame rate..
input_folder_name = 'images'
output_folder_name = 'OutputVideo.mp4'
frame_rate = 50 #fps
images_to_video(input_folder_name , output_folder_name , frame_rate)


fourcc ->  1983148141
writing frames..
done.


# 2 Capturing Images:
 Learn how to capture frames from a webcam connected to your computer and save them as images in a folder. You may use either the built-in camera of your laptop or an external one connected through USB.You should also be able to display the frames (the video) on the screen while capturing

In [16]:
def capture_frame_from_camera(cam_port, output_dir):
    cam = cv2.VideoCapture(cam_port)
    if not cam.isOpened():
        print(" camera is not opening..")
        return
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    frame_id = 1
    while True:
        retVal,image = cam.read()
        if retVal:
            cv2.imshow("Streaming frames",image)
            inp = cv2.waitKey(1) & 0xFF
            if inp == ord('s'):
                cv2.imwrite(f"{output_dir}/frame_{frame_id}.jpg",image)
                frame_id += 1
                print("image captured.")
            elif inp == ord('d'):
                break
            else:
                continue
        else:
            print("something went wrong.")
            break
            
    cam.release()
    cv2.destroyAllWindows()
    inp = cv2.waitKey(1) #force quit the window
    print("Capturing done.")

In [17]:
cam_port = 0
output_dir = "camera_frames"
capture_frame_from_camera(cam_port, output_dir)

image captured.
image captured.
Capturing done.


Chroma Keying: Read about the technique of chroma keying. Following are a few good starting points: • Introduction: http://en.wikipedia.org/wiki/Chroma key
• Alvy Ray Smith and James F Blinn, ”Blue Screen Matting”, SIGGRAPH’96.
Create an interesting composite of two videos using this technique, possibly with one video including yourselves.

In [ ]:


def read_video(video_path):
    video = cv2.VideoCapture(video_path)
    return video

def resize_frame(frame, width, height):
    return cv2.resize(frame, (width, height))

def chroma_key(frame, background_frame):
    # Convert the frame to HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Define the range of green color in HSV
    lower_green = np.array([40, 40, 40])
    upper_green = np.array([80, 255, 255])

    # Create a mask for the green color
    mask = cv2.inRange(hsv, lower_green, upper_green)

    # Invert the mask to get the foreground (subject)
    fg_masked = cv2.bitwise_and(frame, frame, mask=~mask)

    # Use the mask to get the background
    bg_masked = cv2.bitwise_and(background_frame, background_frame, mask=mask)

    # Combine the foreground and background
    result = cv2.add(fg_masked, bg_masked)

    return result

def write_video(output_path, frames, fps, width, height):
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    for frame in frames:
        out.write(frame)

    out.release()

def chroma_keying(foreground_path, background_path, output_path):
    foreground_video = read_video(foreground_path)

    # Get video properties (width, height, frames per second) of the foreground video
    fg_width = int(foreground_video.get(3))
    fg_height = int(foreground_video.get(4))
    fps = foreground_video.get(5)

    background_video = read_video(background_path)
    # Get video properties (width, height) of the background video
    bg_width = int(background_video.get(3))
    bg_height = int(background_video.get(4))

    # Create a list to store the processed frames
    processed_frames = []

    while True:
        # Read frames from the videos
        ret_fg, fg_frame = foreground_video.read()
        ret_bg, bg_frame = background_video.read()

        # Break the loop if either of the videos ends
        if fg_frame is None or bg_frame is None:
            break

        # Resize the foreground frame to match the background size
        fg_frame_resized = resize_frame(fg_frame, bg_width, bg_height)

        # Apply chroma keying
        result_frame = chroma_key(fg_frame_resized, bg_frame)

        # Append the processed frame to the list
        processed_frames.append(result_frame)

    # Release the video capture objects
    background_video.release()
    foreground_video.release()
    

    # Write the result to the output video
    write_video(output_path, processed_frames, fps, bg_width, bg_height)

    print("Done.")


foreground_path = "chromaKeying/fg_video.mp4"
background_path = "chromaKeying/bg_video.mp4"
output_path = "chromaKeying/output_video.mp4"

chroma_keying(foreground_path, background_path, output_path)